# Amazon Bedrock - 다른 리전 추론하기

## Overview

이 노트북은 아마존 베드락을 ap-northeast-2 (서울리전)으로 사용할 때, 서울리전에서 지원하지 않는 모델을 사용해야 할 경우 참고할 수 있는 코드를 제공합니다.

## Use case 

- 개발이 특정 리전으로 되어 있고, 타리전의 베드락 모델을 사용해야 하는 경우

## Prerequisites

1. AWS 계정이 생성되었는지 확인합니다.: [Link](https://docs.aws.amazon.com/accounts/latest/reference/manage-acct-creating.html)
2. 사용자에게 Amazon Bedrock에서 올바른 모델에 액세스할 수 있는 권한이 있는지 확인합니다.: [Link](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
3. 이 노트북은 AWS 자격 증명을 설정하여 진행했습니다.

## Notebook Code with Comments

### Setting up the Bedrock clients

첫 번째 셀에서 필요한 라이브러리를 가져와서 Amazon Bedrock용 `boto3` 클라이언트를 생성할 준비를 합니다.


In [4]:
# Import necessary libraries
import os
import json
import boto3

print(f"Boto3 Version: {boto3.__version__}")

session = boto3.Session()
current_region = session.region_name
print(f"Current Region: {current_region}")

Boto3 Version: 1.35.90
Current Region: ap-northeast-2


### Building Amazon Bedrock inference function

모델에 적합한 추론 파라미터를 사용하여 Amazon Bedrock을 호출하는 함수를 빌드합니다.
특정 모델(amazon.nova-pro-v1:0)을 사용할 때만, 특정 리전(us-east-1)을 사용하게 설정합니다. 또한, 실제 어떤 리전에 접근했는지를 확인합니다. 

In [10]:
def get_text_response(model, input_content):
    try:

        # 소스 리전 설정
        source_region = 'us-east-1' if model == "amazon.nova-pro-v1:0" else 'ap-northeast-2'
        
        client = boto3.client(
            'bedrock-runtime',
            region_name=source_region
        )
        
        # 현재 리전 확인을 위한 STS 클라이언트
        sts_client = boto3.client('sts')
        cr_region = sts_client.meta.region_name

        message = {
            "role": "user",
            "content": [{"text": input_content}]
        }
        
        response = client.converse(
            modelId=model,
            messages=[message],
            inferenceConfig={
                "maxTokens": 2000,
                "temperature": 0,
                "topP": 0.9,
                "stopSequences": []
            }
        )
        
        # 리전 정보 추가
        response['region_info'] = {
            'cr_region': cr_region,
            'source_region': source_region
        }
        return response
        
    except Exception as e:
        raise Exception(f"Error calling Bedrock: {str(e)}")


### Inferencing with difference model_id

Amazon Nova 모델은 현재 서울 리전이 지원되지 않기 때문에 이 모델을 사용할 경우, 소스 리전은 버지니어 리전으로 API가 호출되는 것을 확인할 수 있습니다.

In [11]:
from pprint import pprint

output = get_text_response("amazon.nova-pro-v1:0", "오늘 날씨는?")
print("\n리전 정보:")
print(f"현재 리전: {output['region_info']['cr_region']}")
print(f"소스 리전: {output['region_info']['source_region']}")
print("\n전체 응답:")
pprint(output)



리전 정보:
현재 리전: ap-northeast-2
소스 리전: us-east-1

전체 응답:
{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1390',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 02 Jan 2025 15:12:53 GMT',
                                      'x-amzn-requestid': '02ab8c87-8b2a-4253-9084-b0191f40898f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '02ab8c87-8b2a-4253-9084-b0191f40898f',
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 7371},
 'output': {'message': {'content': [{'text': '오늘 날씨에 대한 정보는 실시간으로 변동할 수 있기 '
                                             '때문에, 구체적인 날씨 정보를 제공하기 위해서는 현재 '
                                             '위치와 시간을 기준으로 한 데이터가 필요합니다. 일반적으로 '
                                             '날씨 정보를 얻는 몇 가지 방법은 다음과 같습니다:\n'
                                             '\n'
   

Anthropic사의 Claude 3 Haiku 모델은 현재 서울 리전이 지원되기 때문에 이 모델을 사용할 경우, 소스 리전도 동일하게 서울 리전으로 API가 호출되는 것을 확인할 수 있습니다.

In [12]:
from pprint import pprint

output = get_text_response("anthropic.claude-3-haiku-20240307-v1:0", "오늘 날씨는?")
print("\n리전 정보:")
print(f"현재 리전: {output['region_info']['cr_region']}")
print(f"소스 리전: {output['region_info']['source_region']}")
print("\n전체 응답:")
pprint(output)



리전 정보:
현재 리전: ap-northeast-2
소스 리전: ap-northeast-2

전체 응답:
{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '538',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 02 Jan 2025 15:13:39 GMT',
                                      'x-amzn-requestid': '4c9e89df-a30c-4fa8-994f-42e4907968d5'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4c9e89df-a30c-4fa8-994f-42e4907968d5',
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 1351},
 'output': {'message': {'content': [{'text': '죄송합니다. 제가 현재 날씨 정보를 알 수 있는 방법이 '
                                             '없어서 정확한 날씨를 알려드리기 어렵습니다. 날씨는 '
                                             '지역마다 다를 수 있기 때문에 사용자 분의 현재 위치를 '
                                             '알아야 정확한 날씨 정보를 제공할 수 있습니다. 혹시 '
                                             '사용자 

# END OF NOTEBOOK 